##### Investigate 3298 null values in response_correctness

- Check correlations
- Missing at random?

In [ ]:
associations( dfFlt.filter(F.col('response_correctness').isNull() ).select(*binaryVars, *nominalVars).toPandas(), nan_replace_value='null', figsize=[10,10] )

- Single values
  - scoring_type_code
  - item_is_offline_scored
- Pefect correlation
  - is_affecting_grade & ced_assignment_type_code (previously existed)
  - assigned_item_status & item_type_code_name (previously existed)

In [ ]:
associations( dfFlt.filter(F.col('response_correctness').isNull() ).select(*continousVars).toPandas(), nan_replace_value='null' )

- Single values
  - assignment_attempt_number
  - assignment_max_attempts
- High Correlation
  - final_score_unweighted & points_possible_unweighted

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, sharey=True)

dfFlt.filter(F.col('response_correctness').isNull() ).toPandas().plot.scatter('final_score_unweighted', 'points_possible_unweighted', title='Null', ax=ax1)
dfFlt.sample(False, .001, 8764664).toPandas().plot.scatter('final_score_unweighted', 'points_possible_unweighted', title='Random', ax=ax2)
plt.suptitle('response_correctness vs scores')

plt.show()

- Null pattern similar to random

In [ ]:

fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(15,15))

corr, ax1 = associations( dfFlt.filter(F.col('response_correctness').isNull() ).select(* (F.unix_timestamp(c).alias(c) for c in intervalVars) ).toPandas(), figsize=[10,10], ax=ax1 )

corr, ax2 = associations( dfFlt.select(* (F.unix_timestamp(c).alias(c) for c in intervalVars) ).toPandas(), ax=ax2 )
#plt.suptitle('response_correctness vs scores')

plt.show()



- New Strong correlations
  - max and min_student_stop_datetime & assignment_due_date
  - max and min_student_stop_datetime & assignment_final_submission_date

In [ ]:
# Previous
associations( dfFlt.select(* (F.unix_timestamp(c).alias(c) for c in intervalVars) ).toPandas(), figsize=[10,10] )

In [ ]:
# null
associations( dfFlt.filter(F.col('response_correctness').isNull() ).select('org_new', 'section_new', 'learner_new', 'assessment_new').toPandas(), nan_replace_value='null' )

- Looking for single values
  - none found

###### Check Scores Associations with response_correctness

In [ ]:
def assoc_resp_cor():
  pdDf = dfInv.select('response_correctness','final_score_unweighted').toPandas()
  pdDf['final_score_unweighted'] = np.where(pdDf['final_score_unweighted'] == 0, 'zero', 'nonzero')

  return pd.crosstab(pdDf.final_score_unweighted.fillna('null'), pdDf.response_correctness.fillna('null'))

assoc_resp_cor()

- response_correctness = 'correct' when score is nonzero
- impute response_correctness based on final_score_unweighted with median
  - final_score_unweighted is right skewed
  -

###### Median final_score_unweighted by response_correctness

In [ ]:
def score_by_correct():
  pdDf = dfInv.select('response_correctness','final_score_unweighted').toPandas()
  return pdDf.groupby('response_correctness')[['final_score_unweighted']].median()

score_by_correct()

In [ ]:
# +--------------------+-----+
# |response_correctness|count|
# +--------------------+-----+
# |correct             |43761|
# |incorrect           |30496|
# |null                |3298 |
# |[unassigned]        |1566 |
# |partially_correct   |1427 |
# +--------------------+-----+

pdDf = dfFlt.select('response_correctness', 'final_score_unweighted').toPandas()
pdDf[ pdDf['response_correctness'] == 'correct']['final_score_unweighted'].median()
pdDf[ pdDf['response_correctness'] == 'incorrect']['final_score_unweighted'].median()


In [ ]:

# Bin the data frame by "a" with 10 bins...
bins = np.linspace(pdDf.final_score_unweighted.min(), pdDf.final_score_unweighted.max(), 10)
groups = pdDf.groupby(pd.cut(pdDf.final_score_unweighted, bins))
groups

###### Display Median by response_correctness levels

In [ ]:
  spark.sql("SELECT source, percentile_approx(value, 0.5) FROM raw_data GROUP BY source")

- Single values
  - item_is_offline_scored
  - scoring_type_code
- Correlations
  - is_affecting_grade & ced_assignment_type_code
  - assigned_item_status & item_type_code_name

###### Display Correlated Values of response_correctness = Null

In [ ]:
dfInv.filter(F.col("response_correctness").isNull() == True).select('item_is_offline_scored', 'scoring_type_code', 'is_affecting_grade', 'ced_assignment_type_code', 'assigned_item_status', 'item_type_code_name').distinct().show()


Explanation Null values in response_correctness Perfect Correlations
- response_correctness is null when not scored (scoring_type_code = '[unassigned]')
  - manually scorred
- contridicts item_is_offline_scored = false
  - automatically scorred
- All perfectly correlated fields appear to be default values when not scorred

###### Statistics of final_score_unweighted to response_correctness = Null

In [ ]:
def null_stats():
  dfNull = dfInv.filter(F.col("response_correctness").isNull() == True)

  dfInv.select('final_score_unweighted').toPandas().hist(bins=60)
  plt.suptitle('All Observations')
  plt.title('final_score_unweighted')

  dfInv.select("points_possible_unweighted").toPandas().hist(bins=60)
  plt.suptitle('All Observations')
  plt.title('points_possible_unweighted')

  dfNull.select("final_score_unweighted").toPandas().hist(bins=60)
  plt.suptitle('Null Observations')
  plt.title('final_score_unweighted')

  dfNull.select("points_possible_unweighted").toPandas().hist(bins=60)
  plt.suptitle('Null Observations')
  plt.title('points_possible_unweighted')


null_stats()

- closely correlated with final_score_unweighted = 0

In [ ]:
finish_todo('Investigate 3298 null values in response_correctness')